In [31]:
import main_module as m
import zoho_filter_module as fil
import pandas as pd
import re
import numpy as np

%load_ext autoreload
%autoreload 2

dir_path = r'C:\Users\fajar\Documents\Python\Data\Zoho_Backup'
file_name = '\Leads_'

cols = ['Record Id',	
'Created Time',
'Created Date',	
'First Name',	
'Last Name',	
'Email',	
'Phone',	
'Mobile',	
'Birthdate',	
'Year of Birth',
'Street',	
'Street2',	
'City',	
'State',	
'Country',	
'Nationality',
'Booking Status', 
'BookRef',
'Departure Date',	
'Arrival Date',	
'Card Type',	
'Marital Status',	
'Lead Brand',	
'Lead Sub-Brand',	
'Lead Source',	
'Lead Locations',	
'Lead Source Description',	
'Lead Regions',
'Lead Status',
'Email Opt Out',	
'Opt In']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
df = m.load_data_multiple(dir_path, file_name, cols)
count_data_awal = df.shape[0]
df = df.reindex(columns=cols)
df.shape[0]

4600000

In [33]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='BOOKING_BGAU_BGID_KE')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [34]:
zoho_filter = fil.user_req(df)
zoho_df = df.loc[zoho_filter].copy() # menggunakan copy untuk menghindari setting copy warning
zoho_df.reset_index(drop=True, inplace=True)
count_filter_awal = zoho_df.shape[0]

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin

zoho_df.fillna('empty', inplace=True)
zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Email Opt Out,Opt In
0,zcrm_3584690000000404058,2019-01-07 15:08:38,2019-01-06,Nicholas,Wood,1230woodnicholas@gmail.com,61438600845,empty,empty,empty,...,empty,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,SEAP,New Lead,false,false
1,zcrm_3584690000000404059,2019-01-07 15:08:38,2019-01-06,Liao,Yuan,158066546@qq.com,8618718399923,empty,empty,empty,...,empty,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,Other,New Lead,false,false
2,zcrm_3584690000000404060,2019-01-07 15:08:38,2019-01-06,Vedanta,Abhiyoga,abhiyoga.vedanta@gmail.com,6282146524550,empty,2000-07-05,2000,...,Pelajar,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,SEAP,New Lead,false,false
3,zcrm_3584690000000404065,2019-01-07 15:08:38,2019-01-06,Aidai,Atakanova,aidai.nurbek@gmail.com,996779641590,empty,1997-02-05,1997,...,Single,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,UK/Europe,New Lead,false,false
4,zcrm_3584690000000404068,2019-01-07 15:08:38,2019-01-06,Ardra,Ardra,aliefardraqariru@gmail.com,+6282139300818,+6282139300818,2005-01-20,2005,...,OTHER,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,WIFI USER ID 13494142,SEAP,New Lead,false,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3806665,zcrm_3584690000424091281,2023-01-23 17:34:08,empty,Dominique,Beasley,dombeasley@icloud.com,+61431031168,empty,empty,empty,...,empty,Timeshare Marketing,Other,Wifi,Purple Wifi,WIFI USER ID 29940918,empty,New Lead,false,false
3806666,zcrm_3584690000424093003,2023-01-23 17:03:18,empty,Siti,Nur laela,siti.nurlaela.1981@gmail.com,+6285811607900,+6285811607900,1981-10-10,1981,...,Married,Timeshare Marketing,Karma Experience,Facebook Ads,General,ID | FB LP | Salak | 1st Anniv,SEAP,Hot Prospect,false,false
3806667,zcrm_3584690000424093029,2023-01-23 18:34:15,empty,Syaefulloh,Ramadhan,Syaefullohramadhan20@gmail.com,+6289533354422,+6289533354422,1989-04-20,1989,...,Married,Timeshare Marketing,Karma Experience,Facebook Ads,General,ID | FB LP | Salak | 1st Anniv,SEAP,Hot Prospect,false,false
3806668,zcrm_3584690000424094389,2023-01-23 17:44:38,empty,Padmavathi,Vadada,padma.vadada@gmail.com,918522960425,empty,empty,empty,...,Married,Timeshare Marketing,Karma Club Experience IN,Web Enquiry,General,IND | Web enquiry | Redeem Voucher,India,empty,false,true


# ====================== Awal Filter Umum ====================== 

In [35]:
# data phone, mobile dam email member yang ada di master data (wash member)

m.clean_number(zoho_df, 'Phone')
m.clean_number(zoho_df, 'Mobile')
m.replace_not_valid_to_empty(zoho_df, 'Phone')
m.replace_not_valid_to_empty(zoho_df, 'Mobile')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')

count_deleted_wrong_fromat = m.delete_email_wrong_format(zoho_df, 'Email')
count_after_deleted_wrong_format = zoho_df.shape[0]

In [36]:
# zoho_df.to_excel(r'C:\Users\fajar\Documents\Python\Data\before_clean_OTA.xlsx', index=False)

In [37]:
# Cek leads test
test_leads = zoho_df.loc[
    
        zoho_df['Email'].str.contains('^test@', flags=re.I, regex=True) |
        zoho_df['Email'].str.contains('\+test', flags=re.I, regex=True) |
    
        zoho_df['First Name'].str.contains('^test$', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('^test$', flags=re.I, regex=True) |

        zoho_df['First Name'].str.contains('test 1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test 2', flags=re.I, regex=True)|
        zoho_df['First Name'].str.contains('test1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test2', flags=re.I, regex=True)|

        zoho_df['First Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test$', flags=re.I, regex=True)
    
    
    ]

count_leads_test = test_leads.shape[0]

test_leads.to_excel(r'C:\Users\fajar\Documents\Python\Data\test_leads.xlsx', index=False)
test_leads

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Email Opt Out,Opt In
56732,zcrm_3584690000006082758,2019-04-02 03:11:29,2015-05-12,Michael,Angus,test@hotmail.com,6666666,empty,empty,empty,...,empty,Timeshare Marketing,BGAU,Edm,empty,BaliGetaway AUS Email - J4 - VO_BMG,SEAP,empty,false,false
56733,zcrm_3584690000006082759,2019-04-02 03:11:29,2015-05-12,Hosanna,Hosanna,test@mail.com,6117324567,9718754212356,1989-09-11,1989,...,empty,Timeshare Marketing,BGAU,Edm,empty,BGAU - Digital - EDM - w2apr17 - Fave Hotel Um...,Other,empty,false,false
56734,zcrm_3584690000006082765,2019-04-02 03:11:29,2015-05-12,empty,empty,test@thebaligetaway.com,40556801,empty,empty,empty,...,empty,Timeshare Marketing,BGAU,Survey,empty,Probably the Best Holiday Offer in the World,SEAP,empty,false,false
269179,zcrm_3584690000006547375,2019-04-02 04:07:53,2017-01-31,Nishant,Sharma,test@naukri.com,empty,empty,empty,empty,...,empty,Timeshare Marketing,Karma Royal,Karma Royal Group,empty,Karma list email for Bhanu - International,India,empty,false,false
269180,zcrm_3584690000006547376,2019-04-02 04:07:53,2015-11-05,empty,empty,test@submitemail.com.fr,81928283,empty,empty,empty,...,Married,Timeshare Marketing,Karma Royal,Karma Royal Experience,empty,KRE - Karma Rottnest,SEAP,empty,false,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3802586,zcrm_3584690000423589717,2023-01-19 17:44:28,empty,Iyar,test,ikhyar.setya@karmagroup.com,61789456123,empty,1963-02-14,1963,...,Married,Timeshare Marketing,BGAU,Facebook Ads,General,AU | FB LP | Eden Hotel,SEAP,Hot Prospect,false,false
3803162,zcrm_3584690000423637183,2023-01-19 11:10:38,empty,eka,test,eka.suryawan+10920191@karmagroup.com,6285551627161,6285551627161,1987-07-22,1987,...,Married,Timeshare Marketing,Karma Experience,Facebook Ads,General,KEID | FB Ads | w3jan23 | Karma Salak Cimory R...,SEAP,Hot Prospect,false,false
3803733,zcrm_3584690000423670274,2023-01-19 11:16:44,empty,eka,test222,eka.suryawan+1222@karmagroup.com,6279911001901,6279911001901,1983-08-31,1983,...,Married,Timeshare Marketing,Karma Experience,Facebook Ads,General,KEID | FB Ads | w3jan23 | Karma Salak Cimory R...,SEAP,Hot Prospect,false,false
3805958,zcrm_3584690000423921825,2023-01-23 13:51:14,empty,iyar,test,ikhyar.setya+test@karmagroup.com,6289685194646,6289685194646,1992-06-25,1992,...,Single,Timeshare Marketing,Karma Experience,Facebook Ads,General,KEID | FB Ads | w3jan23 | Karma Salak Cimory R...,SEAP,Not Qualified,false,false


In [38]:
zoho_df.drop(test_leads.index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_dedup_test = zoho_df.shape[0]

In [39]:
# # Cek Email OTA
# email_OTA = m.find_OTA(zoho_df, 'Email')
# count_email_OTA = email_OTA.shape[0]
# # email_OTA.to_excel(r'C:\Users\fajar\Documents\Python\Data\email_OTA.xlsx', index=False)

# email_OTA

In [40]:
# # delete email OTA
# count_after_clean_OTA = m.delete_OTA(zoho_df, 'Email')
# # zoho_df.to_excel(r'C:\Users\fajar\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

# zoho_df

In [41]:
# Cek Email Karma
email_karma = m.find_karma(zoho_df, 'Email')
count_email_karma = email_karma.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar\Documents\Python\Data\email_OTA.xlsx', index=False)

email_karma

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Email Opt Out,Opt In
21352,zcrm_3584690000002492722,2019-02-22 14:51:05,empty,Reza,Bintang Aurora,reza.bintang@thebaligetaway.com,628113961761,empty,empty,1980,...,empty,Timeshare Marketing,BGID,Facebook Ads,empty,empty,SEAP,empty,false,false
85776,zcrm_3584690000006152415,2019-04-02 03:17:56,2015-03-11,Mark,Townsend,marktw@karma.co.nz,empty,empty,empty,empty,...,empty,Timeshare Marketing,BGAU,Prospect,empty,empty,SEAP,empty,false,false
91802,zcrm_3584690000006167006,2019-04-02 03:19:20,2015-05-12,empty,empty,lee+baligetawaycomau@ccp.com.au,61400481632,empty,empty,empty,...,empty,Timeshare Marketing,BGAU,Facebook Ads,empty,BGAU - FB - Ion 2,SEAP,empty,false,false
99206,zcrm_3584690000006184540,2019-04-02 03:21:02,2017-02-13,Shally,Shally,shally@thebaligetaway.com,empty,empty,empty,empty,...,empty,Timeshare Marketing,BGAU,Facebook Ads,empty,BGAU - Digital - FB - w2feb17 - Neo Hotel Kuta,SEAP,empty,false,false
114916,zcrm_3584690000006220962,2019-04-02 03:24:41,2015-03-10,Mark,Burkster,mark.burke@thebaligetaway.com,6199999999,12152266666,empty,empty,...,empty,Timeshare Marketing,BGAU,Survey,empty,TTM Survey Form - JIMBARAN,SEAP,empty,false,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3805119,zcrm_3584690000423852405,2023-01-22 08:44:59,empty,Rafik,Khetiv,rafik.khetiv@karmagroup.com,6281237935852,empty,empty,empty,...,empty,Timeshare Marketing,Other,Wifi,Purple Wifi,WIFI USER ID 16893759,empty,New Lead,false,false
3805124,zcrm_3584690000423852575,2023-01-22 13:34:30,empty,I Kadek,Edi Sucitra,edi.sucitra@karmagroup.com,6289688605922,empty,empty,empty,...,empty,Timeshare Marketing,Other,Wifi,Purple Wifi,WIFI USER ID 29181923,empty,New Lead,false,false
3805283,zcrm_3584690000423862353,2023-01-22 15:41:49,empty,I Kadek,Edi Sucitra,edi.sucitra@karmagroup.com,6289688605922,empty,empty,empty,...,empty,Timeshare Marketing,Other,Wifi,Purple Wifi,WIFI USER ID 29181923,empty,New Lead,false,false
3805560,zcrm_3584690000423902533,2023-01-23 13:05:35,empty,Rafik,Khetiv,rafik.khetiv@karmagroup.com,6281237935852,empty,empty,empty,...,empty,Timeshare Marketing,Other,Wifi,Purple Wifi,WIFI USER ID 16893759,empty,New Lead,false,false


In [42]:
# delete email Karma
count_after_clean_karma = m.delete_karma(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Email Opt Out,Opt In
0,zcrm_3584690000000404058,2019-01-07 15:08:38,2019-01-06,Nicholas,Wood,1230woodnicholas@gmail.com,61438600845,empty,empty,empty,...,empty,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,SEAP,New Lead,false,false
1,zcrm_3584690000000404059,2019-01-07 15:08:38,2019-01-06,Liao,Yuan,158066546@qq.com,8618718399923,empty,empty,empty,...,empty,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,Other,New Lead,false,false
2,zcrm_3584690000000404060,2019-01-07 15:08:38,2019-01-06,Vedanta,Abhiyoga,abhiyoga.vedanta@gmail.com,6282146524550,empty,2000-07-05,2000,...,Pelajar,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,SEAP,New Lead,false,false
3,zcrm_3584690000000404065,2019-01-07 15:08:38,2019-01-06,Aidai,Atakanova,aidai.nurbek@gmail.com,996779641590,empty,1997-02-05,1997,...,Single,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,UK/Europe,New Lead,false,false
4,zcrm_3584690000000404068,2019-01-07 15:08:38,2019-01-06,Ardra,Ardra,aliefardraqariru@gmail.com,6282139300818,6282139300818,2005-01-20,2005,...,OTHER,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,WIFI USER ID 13494142,SEAP,New Lead,false,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3806007,zcrm_3584690000424088106,2023-01-23 18:45:57,empty,Marganda Parulian,Siregar,shardtaty@gmail.com,6281360030016,6281360030016,1991-11-26,1991,...,Married,Timeshare Marketing,BGID,Facebook Ads,Hotel Partner,ID | FB LP | Romantic Getaway,SEAP,Hot Prospect,false,false
3806008,zcrm_3584690000424091281,2023-01-23 17:34:08,empty,Dominique,Beasley,dombeasley@icloud.com,61431031168,empty,empty,empty,...,empty,Timeshare Marketing,Other,Wifi,Purple Wifi,WIFI USER ID 29940918,empty,New Lead,false,false
3806009,zcrm_3584690000424093003,2023-01-23 17:03:18,empty,Siti,Nur laela,siti.nurlaela.1981@gmail.com,6285811607900,6285811607900,1981-10-10,1981,...,Married,Timeshare Marketing,Karma Experience,Facebook Ads,General,ID | FB LP | Salak | 1st Anniv,SEAP,Hot Prospect,false,false
3806010,zcrm_3584690000424093029,2023-01-23 18:34:15,empty,Syaefulloh,Ramadhan,syaefullohramadhan20@gmail.com,6289533354422,6289533354422,1989-04-20,1989,...,Married,Timeshare Marketing,Karma Experience,Facebook Ads,General,ID | FB LP | Salak | 1st Anniv,SEAP,Hot Prospect,false,false


In [43]:
# hitung jumlah duplicate
count_duplicate_general = zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])].shape[0]

zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])]

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Email Opt Out,Opt In
1965,zcrm_3584690000000657400,2019-01-13 16:37:24,empty,Roberto,Halim Wibowo,robertohw139@gmail.com,6281286298908,empty,1980-01-01,1980,...,Married,Timeshare Marketing,BGID,Facebook Ads,empty,empty,SEAP,empty,false,false
2220,zcrm_3584690000000659532,2019-01-14 00:23:28,empty,Rudianto,Rudianto Hernowo,rudiantohernowo16@gmail.com,6281314651098,empty,1953-02-16,1953,...,Married,Timeshare Marketing,BGID,Facebook Ads,empty,empty,SEAP,empty,false,false
4668,zcrm_3584690000000708689,2019-01-17 10:58:55,empty,Edy,Susanto,edysusanto1083@yahoo.com,6281286751055,empty,1983-01-01,1983,...,Single,Timeshare Marketing,BGID,Facebook Ads,empty,empty,SEAP,empty,false,false
6720,zcrm_3584690000000854019,2019-01-19 19:37:55,empty,Tri Dhiangka,Yuldhika,dhika.tridhiangka@gmail.com,6285640234533,empty,1988-01-01,1988,...,Married,Timeshare Marketing,BGID,Web Enquiry,empty,empty,SEAP,empty,false,false
7095,zcrm_3584690000000883613,2019-01-21 17:22:41,empty,Sri,Lestari,arilestari80@gmail.com,6281216998993,empty,1980-01-01,1980,...,Single,Timeshare Marketing,BGID,Web Enquiry,empty,empty,SEAP,empty,false,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3805988,zcrm_3584690000424075110,2023-01-23 17:45:59,empty,Yudha,Ervanes,ervanesyudha@gmail.com,6282233374131,6282233374131,1987-11-28,1987,...,Married,Timeshare Marketing,BGID,Facebook Ads,Hotel Partner,ID | FB LP | Romantic Getaway,SEAP,Hot Prospect,false,false
3805994,zcrm_3584690000424082027,2023-01-23 16:39:14,empty,empty,Ray Thiesfield,rlt56@live.com,61407796810,empty,1956-06-16,empty,...,empty,Timeshare Marketing,BGAU,Facebook Ads,empty,"Week 50 -51, Dec 16 - Dec 22, 2022 | AU | FB A...",empty,empty,false,false
3805998,zcrm_3584690000424086386,2023-01-23 17:31:27,empty,Dante,Spina,dantespina84@gmail.com,61423951245,empty,empty,empty,...,empty,Timeshare Marketing,Other,Wifi,Purple Wifi,WIFI USER ID 12633898,empty,New Lead,false,false
3806000,zcrm_3584690000424087428,2023-01-23 17:29:13,empty,arabella,spina,arabellaspina@icloud.com,61423951246,empty,empty,empty,...,empty,Timeshare Marketing,Other,Wifi,Purple Wifi,WIFI USER ID 29940754,empty,New Lead,false,false


In [44]:
zoho_df.drop_duplicates(subset=['Phone','Mobile','Email'], inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = zoho_df.shape[0]

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Email Opt Out,Opt In
0,zcrm_3584690000000404058,2019-01-07 15:08:38,2019-01-06,Nicholas,Wood,1230woodnicholas@gmail.com,61438600845,empty,empty,empty,...,empty,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,SEAP,New Lead,false,false
1,zcrm_3584690000000404059,2019-01-07 15:08:38,2019-01-06,Liao,Yuan,158066546@qq.com,8618718399923,empty,empty,empty,...,empty,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,Other,New Lead,false,false
2,zcrm_3584690000000404060,2019-01-07 15:08:38,2019-01-06,Vedanta,Abhiyoga,abhiyoga.vedanta@gmail.com,6282146524550,empty,2000-07-05,2000,...,Pelajar,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,SEAP,New Lead,false,false
3,zcrm_3584690000000404065,2019-01-07 15:08:38,2019-01-06,Aidai,Atakanova,aidai.nurbek@gmail.com,996779641590,empty,1997-02-05,1997,...,Single,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,UK/Europe,New Lead,false,false
4,zcrm_3584690000000404068,2019-01-07 15:08:38,2019-01-06,Ardra,Ardra,aliefardraqariru@gmail.com,6282139300818,6282139300818,2005-01-20,2005,...,OTHER,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,WIFI USER ID 13494142,SEAP,New Lead,false,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3516940,zcrm_3584690000424088080,2023-01-23 18:29:11,empty,Dewi mirawati,Dewi mirawati,zidannayla9@gmail.com,625846507754,625846507754,1979-08-30,1979,...,Married,Timeshare Marketing,BGID,Facebook Ads,Hotel Partner,ID | FB LP | SB Legian v4,SEAP,Hot Prospect,false,false
3516941,zcrm_3584690000424088106,2023-01-23 18:45:57,empty,Marganda Parulian,Siregar,shardtaty@gmail.com,6281360030016,6281360030016,1991-11-26,1991,...,Married,Timeshare Marketing,BGID,Facebook Ads,Hotel Partner,ID | FB LP | Romantic Getaway,SEAP,Hot Prospect,false,false
3516942,zcrm_3584690000424093003,2023-01-23 17:03:18,empty,Siti,Nur laela,siti.nurlaela.1981@gmail.com,6285811607900,6285811607900,1981-10-10,1981,...,Married,Timeshare Marketing,Karma Experience,Facebook Ads,General,ID | FB LP | Salak | 1st Anniv,SEAP,Hot Prospect,false,false
3516943,zcrm_3584690000424093029,2023-01-23 18:34:15,empty,Syaefulloh,Ramadhan,syaefullohramadhan20@gmail.com,6289533354422,6289533354422,1989-04-20,1989,...,Married,Timeshare Marketing,Karma Experience,Facebook Ads,General,ID | FB LP | Salak | 1st Anniv,SEAP,Hot Prospect,false,false


In [45]:
# find_member_booking = zoho_df.copy()

# find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
# find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
# find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
# find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
# find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

# find_member_booking.loc[find_member_booking['Phone'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
# find_member_booking.loc[find_member_booking['Mobile'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

# find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

# find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

# count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
# count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

# #find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\booking-member-dedup.csv', index=False)

# #find_member_booking.loc[find_member_booking['Same Value Find'] == 'Booking-Phone1'] 

In [46]:
# data phone, mobile dam email member yang ada di master data 
count_wash_member = zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)

count_after_wash_member = zoho_df.shape[0]


In [47]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Keseluruhan Master Zoho : "+str(count_data_awal))
print("Jumlah Filter Leads Awal : "+str(count_filter_awal))
print("Jumlah Format Email Salah : "+str(count_deleted_wrong_fromat))
print("Jumlah Setelah Clean Format Email Salah : "+str(count_after_deleted_wrong_format))
print("Jumlah Lead Test : "+str(count_leads_test))
print("Jumlah Setelah Clean Lead Test : "+str(count_after_dedup_test))
# print("Jumlah Email OTA : "+str(count_email_OTA))
# print("Jumlah Setelah Clean Email OTA : "+str(count_after_clean_OTA))
print("Jumlah Email Karma : "+str(count_email_karma))
print("Jumlah Setelah Clean Email Karma : "+str(count_after_clean_karma))
print("Jumlah Duplicate (Phone, Mobile dan Email) : "+str(count_duplicate_general))
print("Jumlah Setelah Clean Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Wash Booking dan Member : "+str(count_wash_member))
print("Jumlah Setelah Wash Member : "+str(count_after_wash_member))

Summary
Jumlah Keseluruhan Master Zoho : 4600000
Jumlah Filter Leads Awal : 3806670
Jumlah Format Email Salah : 23
Jumlah Setelah Clean Format Email Salah : 3806647
Jumlah Lead Test : 264
Jumlah Setelah Clean Lead Test : 3806383
Jumlah Email Karma : 371
Jumlah Setelah Clean Email Karma : 3806012
Jumlah Duplicate (Phone, Mobile dan Email) : 289067
Jumlah Setelah Clean Duplicate : 3516945
Jumlah Wash Booking dan Member : 110695
Jumlah Setelah Wash Member : 3406250


In [48]:
zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Marital Status,Lead Brand,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Email Opt Out,Opt In
0,zcrm_3584690000000404058,2019-01-07 15:08:38,2019-01-06,Nicholas,Wood,1230woodnicholas@gmail.com,61438600845,empty,empty,empty,...,empty,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,SEAP,New Lead,false,false
1,zcrm_3584690000000404059,2019-01-07 15:08:38,2019-01-06,Liao,Yuan,158066546@qq.com,8618718399923,empty,empty,empty,...,empty,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,Other,New Lead,false,false
2,zcrm_3584690000000404060,2019-01-07 15:08:38,2019-01-06,Vedanta,Abhiyoga,abhiyoga.vedanta@gmail.com,6282146524550,empty,2000-07-05,2000,...,Pelajar,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,SEAP,New Lead,false,false
3,zcrm_3584690000000404065,2019-01-07 15:08:38,2019-01-06,Aidai,Atakanova,aidai.nurbek@gmail.com,996779641590,empty,1997-02-05,1997,...,Single,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,empty,UK/Europe,New Lead,false,false
4,zcrm_3584690000000404068,2019-01-07 15:08:38,2019-01-06,Ardra,Ardra,aliefardraqariru@gmail.com,6282139300818,6282139300818,2005-01-20,2005,...,OTHER,Timeshare Marketing,Other Bali DB,Wifi,Purple Wifi,WIFI USER ID 13494142,SEAP,New Lead,false,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3516940,zcrm_3584690000424088080,2023-01-23 18:29:11,empty,Dewi mirawati,Dewi mirawati,zidannayla9@gmail.com,625846507754,625846507754,1979-08-30,1979,...,Married,Timeshare Marketing,BGID,Facebook Ads,Hotel Partner,ID | FB LP | SB Legian v4,SEAP,Hot Prospect,false,false
3516941,zcrm_3584690000424088106,2023-01-23 18:45:57,empty,Marganda Parulian,Siregar,shardtaty@gmail.com,6281360030016,6281360030016,1991-11-26,1991,...,Married,Timeshare Marketing,BGID,Facebook Ads,Hotel Partner,ID | FB LP | Romantic Getaway,SEAP,Hot Prospect,false,false
3516942,zcrm_3584690000424093003,2023-01-23 17:03:18,empty,Siti,Nur laela,siti.nurlaela.1981@gmail.com,6285811607900,6285811607900,1981-10-10,1981,...,Married,Timeshare Marketing,Karma Experience,Facebook Ads,General,ID | FB LP | Salak | 1st Anniv,SEAP,Hot Prospect,false,false
3516943,zcrm_3584690000424093029,2023-01-23 18:34:15,empty,Syaefulloh,Ramadhan,syaefullohramadhan20@gmail.com,6289533354422,6289533354422,1989-04-20,1989,...,Married,Timeshare Marketing,Karma Experience,Facebook Ads,General,ID | FB LP | Salak | 1st Anniv,SEAP,Hot Prospect,false,false


# ====================== Akhir Filter Umum ====================== 

In [49]:
zoho_df.replace('empty', '', inplace=True)
# zoho_df.to_excel(r'C:\Users\fajar\Documents\Python\Data\zoho.xlsx', index=False)

In [51]:
row_split = 500000
file_name = 'timeshare_marketing_'
file_type = 'xlsx'

m.split_file(zoho_df, row_split, file_name, file_type)